In [1]:
#from two_stage_rag_pipeline import *
from rag_pipeline import *
#from test_pipeline import *
import os
import pandas as pd
from uuid import uuid4
from tqdm.notebook import tqdm
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall,
)
# 최신 버전 기준 (ragas >= 0.1.2)
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas import EvaluationDataset
from datasets import Dataset as HFDataset
# 초기 세팅
vectorstore = initialize_pinecone()
conversation_chain = initialize_conversation(vectorstore)

model loaded...


In [2]:
# sparse retriever test를 위한 doc_list 생성
from loader.load_documents import *
# 1. 경로 설정
timetable_path = "data/Timetable_Crawling_Data"
guidebook_path = "data/major_guide_2025.pdf"
pram_path = "data/pram_2025_1.json"
prof_path = "data/hufs_professor.json"
college_path = "data/hufs_colleges.json"
notice_path = "data/hufs_notice.json"
schedule_path = "data/hufs_schedule.json"

# 2. 문서 로드
print("강의시간표 로드 중...")
subject_docs = load_all_subject_documents(timetable_path)

print("전공가이드북 로드 중...")
guidebook_docs = load_major_guidebook(guidebook_path)

print("수강편람 로드 중...")
pram_docs = load_sugang_pram(pram_path)

print("교수진 정보 로드 중...")
prof_docs = load_professor_documents(prof_path)

print("단과대 정보 로드 중...")
college_docs = load_college_intro(college_path)

print("학사공지 로드 중...")
notice_docs = load_notice_documents(notice_path)

print("학사일정 로드 중...")
schedule_docs = load_academic_schedule(schedule_path)

doc_list = subject_docs + guidebook_docs + pram_docs + prof_docs + college_docs + notice_docs +schedule_docs

강의시간표 로드 중...
전공가이드북 로드 중...
수강편람 로드 중...
교수진 정보 로드 중...
단과대 정보 로드 중...
학사공지 로드 중...
학사일정 로드 중...


In [3]:
session_id = "test-session"  # 또는 str(uuid4()) 사용

# doc_list가 Document 객체 리스트일 경우, text만 추출
texts = [doc.page_content for doc in doc_list]

# bm25 retriever 초기화
bm25_retriever = BM25Retriever.from_texts(texts)
bm25_retriever.k = 20  # BM25Retriever의 검색 결과 개수

# 실험에 따라 변경될 내용
pinecone_retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# 앙상블 retriever를 초기화.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, pinecone_retriever],
    weights=[0.3, 0.7]
)

In [4]:
import os
from dotenv import load_dotenv
from datasets import Dataset


def main():
    load_dotenv()

    # 저장 폴더 준비
    os.makedirs("data", exist_ok=True)

    # 질문 리스트
    questions = [
        "월요일 7, 8교시에 들을 수 있는 교양 있으면 알려줘.",
        "글스산 졸업하면 뭐해?",
        "졸업하기 전에 무슨 영어성적 제출해야 된다고 했던거 같은데 그건 뭐야?",
        "후기 이중에 대해 설명해줘.",
        "김정환 교수님 연구실 어디야?",
        "컴퓨팅사고를 AI융합전공 이중과목으로 변경해야되는데 이수구분변경 신청 어떻게 해?",
        "감기로 병원다녀왔는데 진료확인서로 유고결석 가능해?",
        ("나 정보통신공학과인데 혹시 졸업시험 대신 졸업논문을 선택해도 될까? "
         "논문을 쓰게 되면 머신러닝 관련해서 작성하고 싶은데 이와 관련해서 상담할 수 있는 교수님이 계시면 이메일 좀 알려줘."),
        "일통 FLEX 시험 점수 제출 기간 알고 싶어.",
        "글스산 세부전공 신청해야되는데, 신청기간이랑 방법 알 수 있어? 그리고 결과는 언제나와?"
    ]

    # Ground‐truth 리스트
    reference = [
        ["월요일 7,8교시에 들을 수 있는 교양 과목에는 '인간과 사회', '대중문화 예술의 이해', '비즈니스협상' 등이 있습니다."],
        ["글로벌스포츠산업학부를 졸업하면 주로 다음과 같은 분야에서 활동하게 됩니다:\n"
         "1. 스포츠매니지먼트 및 이벤트 기획\n"
         "2. 스포츠 마케팅 및 홍보\n"
         "3. 스포츠 데이터 분석가\n"
         "4. 스포츠 미디어\n"
        "이 외에도 졸업 후에는 대학원 진학이나 관련 분야의 연구를 진행할 수 있는 기회도 있습니다."
        ],
        ["졸업하기 전에 제출해야 하는 영어 성적은 외국어 인증을 위한 것으로, 다음과 같은 기준을 충족해야 합니다:\n\n1." 
        "**인증 시험 성적**: 제출 시점 기준 유효기간(2년) 이내의 인증 시험 성적이 필요합니다. 인정되는 시험은 다음과 같습니다:\n"   
        "- TOEIC\n   - TOEFL\n   - IELTS\n   - FLEX 외국어 능력 시험 등\n\n2." 
        "**대체 과정 수료증**: 인증 시험 성적 대신 외국어 연수 평가원 또는 외국어 교육센터에서 제공하는 대체 과정을 이수한 경우," 
        "해당 수료증을 제출할 수 있습니다.\n\n3. **제출 마감일**: 졸업하고자 하는 최종 학기 말까지 반드시 제출해야 하며," 
        "재학 기간 동안 1회 외국어 인증 신청서를 제출할 수 있습니다.\n\n"
        "정확한 기준과 제출 방법은 학사종합지원센터에 문의하시거나, 해당 과목의 담당 교수님께 확인하시기 바랍니다."],
         ["후기 이중전공은 부전공 이수자가 정규 8학기 내에 제1전공 졸업학점(부전공 21학점 포함)을 모두 취득한 후, "
         "1~2학기 추가 수학하여 해당 부전공의 이중전공화에 필요한 나머지 학점을 취득함으로써, "
         "제1전공과 이중전공 학위를 동시에 취득하는 제도입니다.\n"
        "신청은 매년 4월에 한 번만 가능하며, 신청 자격은 현재 7~8학기 이수 중인 학생으로, 부전공을 이수 중인 경우에 해당합니다.\n" 
        "추가학기를 등록해야 하며, 등록금은 후기 이중전공 학과의 등록금으로 납부해야 합니다. 더 궁금하신 점이 있으시면 말씀해 주시기 바랍니다."],
        ["김정환 교수님의 연구실은 어문관 435호에 위치해 있습니다. 추가로 궁금하신 점이 있으시면 언제든지 말씀해 주세요."],
        ["컴퓨팅사고를 AI융합전공 이중과목으로 변경하시려면, 이수구분 변경 신청서를 작성하신 후," 
        "해당 전공 소속 단과대학의 담당 선생님에게 이메일로 제출하시거나 직접 방문하여 제출하셔야 합니다.\n"
        "신청은 졸업학기 졸업사정일 전까지 가능하며, 제출 후 3~4일 이내에 종합정보시스템에서 변경 결과를 확인하실 수 있습니다.\n" 
        "만약 변경이 되지 않았거나 문제가 발생한 경우, 즉시 담당 선생님에게 연락하여 확인하셔야 합니다."
        "자세한 사항은 학사종합지원센터의 공지사항을 참고하시기 바랍니다. 도움이 필요하시면 언제든지 문의해 주세요."],
        ["감기로 병원에 다녀오신 경우, 진료확인서로 유고결석을 인정받기 위해서는 몇 가지 조건이 있습니다."
         "일반적으로 의원에서 발급받은 서류는 유고결석 인정에 해당되지 않으며, 병원급 이상의 의료기관에서 발급된 증빙서류가 필요합니다."
         "따라서, 감기로 병원급 의료기관에서 진료를 받으셨다면 해당 증빙서류를 제출하시면 유고결석이 가능할 수 있습니다.\n"
        "정확한 사항은 담당 교수님께 문의하시고, 필요한 서류를 제출하시는 것이 좋습니다. 추가적인 질문이 있으시면 언제든지 말씀해 주시기 바랍니다."],
        ["졸업시험 대신 졸업논문을 선택하실 수 있습니다."
         "졸업논문을 작성하실 경우, 머신러닝 관련 주제로 진행하고 싶으시다면, 관련 상담을 위해 한희일 교수님께 연락하실 수 있습니다\n"
         "한희일 교수님의 이메일은 hihahn@hufs.ac.kr 입니다."],
        ["졸업시험 점수 제출 기간은 공지사항에 따라 각 학과 사무실로 문의해야 합니다. 추가적인 질문이 있으시면 말씀해주세요."],
        ["글로벌스포츠산업학부의 세부전공 신청에 대한 정보는 다음과 같습니다.\n"
         "신청기간: 2025년 5월 26일(월) 10:00부터 5월 30일(금) 16:00까지입니다. 신청기간 이후에는 추가 신청이나 변경이 절대 불가하니 유의하시기 바랍니다.\n"
         "신청방법: 종합정보시스템에 접속하셔서 전공/교직관리 메뉴에서 1전공 신청을 하시면 됩니다.\n"
         "배정결과: 배정 확정은 2025년 7월 중에 이루어질 예정입니다. 성적 반영이 완료된 후에 결과를 확인하실 수 있습니다.\n"
         "추가적인 문의사항이 있으시면 학사종합지원센터에 연락하시기 바랍니다."]
    ]

    # RAG 추론 및 컨텍스트 수집
    answers = []
    contexts = []
    session_id = "test-session"  # 또는 str(uuid4()) 사용
    
    for q in questions:
        result = conversation_chain.invoke(
            {"input": q},
            config={"configurable": {"session_id": session_id}}
        )
    
        answers.append(result["answer"])  # ⬅️ 문자열만 저장
        docs = ensemble_retriever.invoke(q)
        contexts.append([doc.page_content for doc in docs])
    # Dataset 생성
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "reference": reference
    }
    dataset = Dataset.from_dict(data)


    dataset.save_to_disk("ragas_naive_data")

    print("Dataset saved to data")
    print(dataset)

if __name__ == "__main__":
    main()

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset saved to data
Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 10
})


In [5]:
from datasets import load_from_disk
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    faithfulness,
    answer_relevancy,
)

def main():
    # 저장된 dataset 불러오기
    dataset = load_from_disk("ragas_naive_data")

    def convert_reference(example):
        ref = example["reference"]
        if isinstance(ref, list):
            example["reference"] = ref[0]
        return example
    
    # 3. map으로 전체에 적용
    dataset = dataset.map(convert_reference)

    # 평가 실행
    result = evaluate(
        dataset=dataset,
        metrics=[
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy
        ],
    )

    # 결과 확인 및 저장
    print(result)
    df = result.to_pandas()
    df.to_csv("data/evaluation_results_naive.csv", index=False)
    print("Evaluation results saved to evaluation_results.csv")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Exception raised in Job[34]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-4ntRKl40y51sE7ehbpQaBMEB on tokens per min (TPM): Limit 200000, Used 189372, Requested 13723. Please try again in 928ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[16]: TimeoutError()


{'context_precision': 0.4977, 'context_recall': 0.7017, 'faithfulness': 0.7620, 'answer_relevancy': 0.5864}
Evaluation results saved to evaluation_results.csv


In [11]:
import pandas as pd
df = pd.read_csv('data/evaluation_results_naive.csv')
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,"월요일 7, 8교시에 들을 수 있는 교양 있으면 알려줘.",['제1학기 성적 열람 및 정정 일정:\n2025-06-30 (월요일)\n2025-...,"월요일 7, 8교시에 수강할 수 있는 교양 과목은 다음과 같습니다:\n\n1. **...","월요일 7,8교시에 들을 수 있는 교양 과목에는 '인간과 사회', '대중문화 예술의...",0.000000,0.000000,1.000000,0.798388
1,글스산 졸업하면 뭐해?,['3학년 통번역을 중점적으로 다루며 졸업시험을 위한 FLEX 준비도 시\n작하게...,'글스산'은 '글로벌스포츠산업학부'를 의미합니다. 졸업 후에는 다음과 같은 진로를 ...,글로벌스포츠산업학부를 졸업하면 주로 다음과 같은 분야에서 활동하게 됩니다:\n1. ...,0.307407,1.000000,0.857143,0.766445
2,졸업하기 전에 무슨 영어성적 제출해야 된다고 했던거 같은데 그건 뭐야?,['[글로벌] 2025년 전기(2월 졸업) 졸업예비사정 결과 조회 안내\n\n(* ...,졸업하기 전에 제출해야 하는 영어 성적은 '외국어 인증'과 관련된 것입니다. 졸업을...,"졸업하기 전에 제출해야 하는 영어 성적은 외국어 인증을 위한 것으로, 다음과 같은 ...",0.695620,1.000000,0.909091,0.000000
3,후기 이중에 대해 설명해줘.,"['| 구분 | 기간 | 비고 |', '정하는 시간을 갖게 됩니다.\n2학년 한국...",'후기 이중전공'은 부전공을 이수 중인 학생이 정규 8학기 내에 제1전공 졸업학점(...,후기 이중전공은 부전공 이수자가 정규 8학기 내에 제1전공 졸업학점(부전공 21학점...,0.317260,0.750000,1.000000,0.801661
4,김정환 교수님 연구실 어디야?,['김정환 교수님은 루마니아학과 학과 소속입니다. 교수 직위를 맡고 있습니다. 이메...,김정환 교수님은 루마니아학과 소속입니다. 연구실은 어문관 435호에 위치해 있습니다...,김정환 교수님의 연구실은 어문관 435호에 위치해 있습니다. 추가로 궁금하신 점이 ...,NaN,0.500000,0.750000,0.954682
5,컴퓨팅사고를 AI융합전공 이중과목으로 변경해야되는데 이수구분변경 신청 어떻게 해?,"[""[서울] 이수구분 변경 신청 방법 안내\n\n[서울] 이수구분 변경 신청 방법 ...",'컴퓨팅사고' 과목을 AI융합전공 이중과목으로 변경하기 위해서는 이수구분 변경 신청...,"컴퓨팅사고를 AI융합전공 이중과목으로 변경하시려면, 이수구분 변경 신청서를 작성하신...",0.915715,0.800000,0.875000,0.906840
6,감기로 병원다녀왔는데 진료확인서로 유고결석 가능해?,['이병도 교수님은 태국어통번역학과 학과 소속입니다. 교수 직위를 맡고 있습니다. ...,"감기로 병원에 다녀오신 경우, 일반 의원에서 발급한 진료확인서는 유고결석으로 인정되...","감기로 병원에 다녀오신 경우, 진료확인서로 유고결석을 인정받기 위해서는 몇 가지 조...",0.678284,0.800000,0.800000,0.000000
7,나 정보통신공학과인데 혹시 졸업시험 대신 졸업논문을 선택해도 될까? 논문을 쓰게 되...,['김명진 명예교수님 교수님은 정보통신공학과 학과 소속입니다. 이메일은 mjkim@...,정보통신공학과에서는 졸업시험 대신 졸업논문을 선택할 수 있습니다. 졸업논문을 작성할...,"졸업시험 대신 졸업논문을 선택하실 수 있습니다.졸업논문을 작성하실 경우, 머신러닝 ...",0.453118,0.666667,0.000000,0.855577
8,일통 FLEX 시험 점수 제출 기간 알고 싶어.,['[공통] 2025-1학기(상반기) 교내 FLEX 졸업시험 시행 안내\n\n[공통...,일본어통번역학과(일통)의 FLEX 시험 점수 제출 기간은 해당학과 사무실에 문의하셔...,졸업시험 점수 제출 기간은 공지사항에 따라 각 학과 사무실로 문의해야 합니다. 추가...,0.769505,0.500000,NaN,0.000000
9,"글스산 세부전공 신청해야되는데, 신청기간이랑 방법 알 수 있어? 그리고 결과는 언제나와?","['[글로벌] 이수구분변경 신청 방법 안내\n\n다. 이수구분 신청 전, 전공교류교...",글로벌스포츠산업학부(글스산)의 세부전공 신청은 다음과 같이 진행됩니다:\n\n1. ...,글로벌스포츠산업학부의 세부전공 신청에 대한 정보는 다음과 같습니다.\n신청기간: 2...,0.342519,1.000000,0.666667,0.780600


In [12]:
# 보기 좋게 출력하는 함수
def print_ragas_eval(df):
    for i, row in df.iterrows():
        print(f"--- Row {i} ---")
        print(f"[User Input]\n{row['user_input']}\n")
        print(f"[Response]\n{row['response']}\n")
        #print(f"[Reference]\n{row['reference']}\n")
        print(f"[Faithfulness]: {row['faithfulness']:.3f}")
        print(f"[Answer Relevancy]: {row['answer_relevancy']:.3f}")
        print("=" * 40)

# 사용 예시 (상위 5개만 출력)
print_ragas_eval(df)

--- Row 0 ---
[User Input]
월요일 7, 8교시에 들을 수 있는 교양 있으면 알려줘.

[Response]
월요일 7, 8교시에 수강할 수 있는 교양 과목은 다음과 같습니다:

1. **신화와종교**
   - 교수: 강미라
   - 강의실: 2303-3호
   - 학점: 2

2. **제자백가다시읽기**
   - 교수: 이은주
   - 강의실: 2315호
   - 학점: 2

3. **실용스페인어기초다지기**
   - 교수: 이강국
   - 강의실: 1303호
   - 학점: 2

4. **생명과학과불교철학특강**
   - 교수: 김원명
   - 강의실: 4216호
   - 학점: 3

이 과목들은 전공필수가 아니며, 다양한 학문적 배경을 가진 학생들이 수강할 수 있습니다. 혹시 더 도와드릴까요?

[Faithfulness]: 1.000
[Answer Relevancy]: 0.798
--- Row 1 ---
[User Input]
글스산 졸업하면 뭐해?

[Response]
'글스산'은 '글로벌스포츠산업학부'를 의미합니다. 졸업 후에는 다음과 같은 진로를 선택할 수 있습니다:

1. **스포츠 매니지먼트 및 이벤트 기획**: 스포츠 클럽, 리그, 대회, 이벤트 등을 기획하고 운영하는 직무입니다.
2. **스포츠 마케팅 및 홍보**: 스포츠 산업에서 제품, 서비스, 이벤트 등을 마케팅하고 홍보하는 업무를 수행합니다.
3. **스포츠 데이터 분석가**: 스포츠 경기에서 발생하는 다양한 데이터를 수집하고 분석하여 팀이나 선수의 성과를 향상시키는 데 필요한 정보를 도출하는 직무입니다.
4. **스포츠 미디어**: 스포츠 뉴스, 인터뷰, 해설, 하이라이트, 분석 등 다양한 스포츠 관련 콘텐츠를 제작하고 편집하여 방송, 인터넷, 모바일 등에서 전달하는 업무를 수행합니다.

졸업 후에는 이러한 분야에서 활동하며 스포츠 산업의 다양한 측면에서 경력을 쌓을 수 있습니다. 혹시 더 도와드릴까요?

[Faithfulness]: 0.857
[Answer Rele

In [18]:
print(df['retrieved_contexts'][0])

['제1학기 성적 열람 및 정정 일정:\n2025-06-30 (월요일)\n2025-07-01 (화요일)\n2025-07-02 (수요일)\n2025-07-03 (목요일)\n2025-07-04 (금요일)', '[언어학연구방법론] 4학년 대상 / 교수: 이해윤 / 월요일 7,8교시 / 강의실 4210호 / 학점: 2 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님', '[세계사와한국] 전체 학년 대상 / 교수: 이수기 / 수요일 7,8교시 / 강의실 2311호 / 학점: 2 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님\n\n[신화와종교] 전체 학년 대상 / 교수: 강미라 / 월요일 7,8교시 / 강의실 2303-3호 / 학점: 2 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님\n\n[역사란무엇인가] 전체 학년 대상 / 교수: 서상현 / 월요일 3,4교시 / 강의실 2502호 / 학점: 2 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님\n\n[요가경전강독] 전체 학년 대상 / 교수: 최원호 / 금요일 7,8교시 / 강의실 2313호 / 학점: 2 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님', '[역사란무엇인가] 전체 학년 대상 / 교수: 서상현 / 화요일 7,8교시 / 강의실 2502호 / 학점: 2 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님\n\n[윤리적사고와문제해결] 전체 학년 대상 / 교수: 김신 / 목요일 3,4교시 / 강의실 5208호 / 학점: 2 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님\n\n[종교철학입문] 전체 학년 대상 / 교수: 조명동 / 월요일 7,8교시 / 강의실 2503호 / 학점: 2 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목

In [7]:
# 1. 평균값 구하고 전치
df1 = df.mean(numeric_only=True, skipna=True).reset_index()
df1 = df1.T

# 2. 첫 번째 행을 컬럼명으로 설정
df1.columns = df1.iloc[0]

# 3. 첫 번째 행(이전 헤더로 썼던 행) 제거
df1 = df1.drop(df1.index[0]).reset_index(drop=True)

# 4. 모델 컬럼 추가 및 인덱스 삭제
df1 = df1.reset_index(drop=True)
df1['model'] = 'naive'


In [8]:
df1 

index,context_precision,context_recall,faithfulness,answer_relevancy,model
0,0.497714,0.701667,0.761989,0.586419,naive


In [10]:
df1.to_csv('data/naive.csv',encoding='utf-8',index=False)